<a href="https://colab.research.google.com/github/pedrocivita/superComp_pedrotpc/blob/main/aula12/supercompAtividade12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atividade 12 - Supercomputação - Pedro Civita**

In [1]:
# No Colab, configure a GPU (T4 GPU, etc.) acessando "Runtime" -> "Change Runtime Type" -> GPU.
!nvidia-smi  # Verifique se a GPU está ativada

Fri Sep 20 11:20:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## **Código - Exercicio 1**

In [14]:
%%writefile saxpy.cu

#include <thrust/device_vector.h>  // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/transform.h>      // Inclui a função para transformações ponto a ponto
#include <thrust/random.h>         // Inclui a biblioteca para gerar números aleatórios
#include <iostream>                // Biblioteca padrão para entrada e saída de dados
#include <chrono>                  // Biblioteca para medir tempo de execução
#include <cuda_runtime.h>          // Biblioteca para informações de memória da GPU

// Functor para calcular Saxpy (a * x + y)
struct saxpy
{
    int a;

    // Construtor para inicializar a constante 'a'
    saxpy(int a_) : a(a_) {};

    // Função que será chamada pela Thrust (executada na GPU ou CPU)
    __host__ __device__
    double operator()(const double& x, const double& y) const {
        return a * x + y;  // Cálculo Saxpy
    }
};

// Função para exibir o uso da memória da GPU
void mostrarUsoMemoriaGPU() {
    size_t free_mem, total_mem;
    cudaMemGetInfo(&free_mem, &total_mem);
    std::cout << "Memória total da GPU: " << total_mem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre da GPU: " << free_mem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória usada pela GPU: " << (total_mem - free_mem) / (1024 * 1024) << " MB" << std::endl;
}

int main() {
    const int N = 10;  // Tamanho dos vetores

    // Vetores na GPU
    thrust::device_vector<double> x(N);
    thrust::device_vector<double> y(N);
    thrust::device_vector<double> z(N);  // Vetor de saída

    // Preenche os vetores x e y com valores aleatórios
    thrust::default_random_engine rng;
    thrust::uniform_real_distribution<double> dist(0, 100);

    for (int i = 0; i < N; i++) {
        x[i] = dist(rng);  // Preenche x com valores aleatórios
        y[i] = dist(rng);  // Preenche y com valores aleatórios
    }

    // Escolha o valor da constante 'a'
    int a = 100;

    // Mostra a memória da GPU antes da cópia
    std::cout << "Uso de memória da GPU antes da operação Saxpy:" << std::endl;
    mostrarUsoMemoriaGPU();

    // Medir o tempo de execução da operação Saxpy
    auto start = std::chrono::high_resolution_clock::now();

    // Aplica Saxpy (a * x + y) e armazena o resultado em 'z'
    thrust::transform(x.begin(), x.end(), y.begin(), z.begin(), saxpy(a));

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = end - start;

    // Mostra o tempo de execução da operação
    std::cout << "Tempo de execução da operação Saxpy: " << duration.count() << " segundos" << std::endl;

    // Exibe os vetores
    std::cout << "Vetor X: ";
    for (int i = 0; i < N; i++) {
        std::cout << x[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Vetor Y: ";
    for (int i = 0; i < N; i++) {
        std::cout << y[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Resultado (Z = " << a << " * X + Y): ";
    for (int i = 0; i < N; i++) {
        std::cout << z[i] << " ";
    }
    std::cout << std::endl;

    // Mostra a memória da GPU após a cópia
    std::cout << "Uso de memória da GPU após a operação Saxpy:" << std::endl;
    mostrarUsoMemoriaGPU();

    return 0;
}


Overwriting saxpy.cu


In [15]:
!nvcc -arch=sm_75 -std=c++14 saxpy.cu -o saxpy

In [16]:
!./saxpy

Uso de memória da GPU antes da operação Saxpy:
Memória total da GPU: 15102 MB
Memória livre da GPU: 14995 MB
Memória usada pela GPU: 107 MB
Tempo de execução da operação Saxpy: 3.5688e-05 segundos
Vetor X: 0.00224775 60.1353 96.7956 51.4976 26.2906 8.95478 58.223 59.1919 87.6634 72.6212 
Vetor Y: 8.50324 89.1611 18.969 39.8008 74.3512 56.039 80.9567 51.1713 99.5085 96.6611 
Resultado (Z = 100 * X + Y): 8.72802 6102.69 9698.53 5189.56 2703.41 951.517 5903.25 5970.36 8865.85 7358.78 
Uso de memória da GPU após a operação Saxpy:
Memória total da GPU: 15102 MB
Memória livre da GPU: 14995 MB
Memória usada pela GPU: 107 MB


## **Código - Exercicio 2**

In [26]:
%%writefile magnitude_gpu.cu

#include <thrust/device_vector.h>  // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/transform_reduce.h>  // Inclui a função transform_reduce
#include <thrust/functional.h>  // Inclui operações matemáticas como thrust::plus
#include <cmath>  // Inclui a função sqrt para a raiz quadrada
#include <iostream>
#include <chrono>  // Biblioteca para medir o tempo de execução
#include <cuda_runtime.h>  // Biblioteca para verificar o uso da GPU
#include <random>  // Biblioteca padrão de C++ para geração de números aleatórios

// Functor para elevar ao quadrado
struct square {
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;  // Elevar ao quadrado
    }
};

// Função que calcula a magnitude de um vetor na GPU
float magnitude_gpu(thrust::device_vector<float>& v) {
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);  // Raiz quadrada da soma dos quadrados
}

// Função para verificar o uso de memória da GPU
void check_gpu_memory() {
    size_t free_byte;
    size_t total_byte;
    cudaMemGetInfo(&free_byte, &total_byte);

    float free_mb = (float)free_byte / (1024.0f * 1024.0f);
    float total_mb = (float)total_byte / (1024.0f * 1024.0f);
    float used_mb = total_mb - free_mb;

    std::cout << "Memória total da GPU: " << total_mb << " MB" << std::endl;
    std::cout << "Memória livre da GPU: " << free_mb << " MB" << std::endl;
    std::cout << "Memória usada pela GPU: " << used_mb << " MB" << std::endl;
}

int main() {
    const int N = 1000000;  // Tamanho dos vetores

    // Vetor de floats na GPU (device)
    thrust::device_vector<float> d_v(N);

    // Usando a biblioteca padrão de C++ para gerar números aleatórios
    std::default_random_engine rng;
    std::uniform_real_distribution<float> dist(0, 100);

    // Preenchendo o vetor com valores aleatórios
    for (int i = 0; i < N; ++i) {
        d_v[i] = dist(rng);  // Preenche o vetor com valores aleatórios
    }

    std::cout << "=== Antes da execução ===" << std::endl;
    check_gpu_memory();

    // Medindo o tempo de execução na GPU
    auto start_gpu = std::chrono::high_resolution_clock::now();
    float result_gpu = magnitude_gpu(d_v);
    auto end_gpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_gpu = end_gpu - start_gpu;

    // Exibe o resultado da magnitude e o tempo na GPU
    std::cout << "Magnitude calculada na GPU: " << result_gpu << std::endl;
    std::cout << "Tempo de execução na GPU: " << duration_gpu.count() << " segundos" << std::endl;

    std::cout << "=== Após a execução ===" << std::endl;
    check_gpu_memory();

    return 0;
}


Overwriting magnitude_gpu.cu


In [18]:
%%writefile magnitude_cpu.cpp

#include <iostream>
#include <vector>
#include <cmath>  // Inclui a função sqrt para a raiz quadrada
#include <random>
#include <chrono>  // Biblioteca para medir o tempo de execução

// Função para calcular a magnitude de um vetor na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for (float x : v) {
        sum_of_squares += x * x;
    }
    return std::sqrt(sum_of_squares);  // Raiz quadrada da soma dos quadrados
}

int main() {
    const int N = 1000000;  // Tamanho do vetor

    // Vetor de floats na CPU
    std::vector<float> v(N);

    // Preenchendo o vetor com valores aleatórios
    std::default_random_engine rng;
    std::uniform_real_distribution<float> dist(0, 100);
    for (int i = 0; i < N; ++i) {
        v[i] = dist(rng);  // Preenche o vetor com valores aleatórios
    }

    // Medindo o tempo de execução na CPU
    auto start_cpu = std::chrono::high_resolution_clock::now();
    float result_cpu = magnitude_cpu(v);
    auto end_cpu = std::chrono::high_resolution_clock::now();
    std::chrono::duration<float> duration_cpu = end_cpu - start_cpu;

    // Exibe o resultado da magnitude e o tempo na CPU
    std::cout << "Magnitude calculada na CPU: " << result_cpu << std::endl;
    std::cout << "Tempo de execução na CPU: " << duration_cpu.count() << " segundos" << std::endl;

    return 0;
}


Writing magnitude_cpu.cpp


In [27]:
# Compilando e executando o código na GPU
!nvcc -arch=sm_75 -std=c++14 magnitude_gpu.cu -o magnitude_gpu

In [28]:
!./magnitude_gpu

=== Antes da execução ===
Memória total da GPU: 15102.1 MB
Memória livre da GPU: 14993.1 MB
Memória usada pela GPU: 109 MB
Magnitude calculada na GPU: 57730.2
Tempo de execução na GPU: 0.000445143 segundos
=== Após a execução ===
Memória total da GPU: 15102.1 MB
Memória livre da GPU: 14993.1 MB
Memória usada pela GPU: 109 MB


In [24]:
!g++ -std=c++14 magnitude_cpu.cpp -o magnitude_cpu

In [25]:
!./magnitude_cpu

Magnitude calculada na CPU: 57719.4
Tempo de execução na CPU: 0.0109731 segundos


## **Exercício 3**

In [31]:
%%writefile variance_fusion.cu

#include <thrust/device_vector.h>
#include <thrust/reduce.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <chrono>
#include <cuda_runtime.h>

// Functor para a operação de cálculo da variância
struct variance_op {
    float mean;

    variance_op(float mean_) : mean(mean_) {}  // Construtor que armazena a média

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;  // Diferença em relação à média
        return diff * diff;  // Elevar ao quadrado a diferença
    }
};

// Functor para a operação de elevação ao quadrado
struct square_diff {
    float mean;
    square_diff(float mean_) : mean(mean_) {}

    __host__ __device__
    float operator()(float x) const {
        return (x - mean) * (x - mean);
    }
};

// Função para calcular a variância com Fusion Kernel
float calculate_variance_fusion(const thrust::device_vector<float>& d_vec, float mean) {
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

// Função para calcular a média
float calculate_mean(const thrust::device_vector<float>& d_vec) {
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    return sum / d_vec.size();
}

// Função para calcular a variância sem Fusion Kernel (em etapas separadas)
float calculate_variance_separate(const thrust::device_vector<float>& d_vec, float mean) {
    thrust::device_vector<float> temp(d_vec.size());
    thrust::transform(d_vec.begin(), d_vec.end(), temp.begin(), square_diff(mean));

    float variance = thrust::reduce(temp.begin(), temp.end(), 0.0f, thrust::plus<float>()) / d_vec.size();
    return variance;
}

// Função para preencher o vetor com valores
void fill_vector(thrust::device_vector<float>& d_vec) {
    for (int i = 0; i < d_vec.size(); ++i) {
        d_vec[i] = static_cast<float>(i + 1);  // Preenche com valores simples para teste
    }
}

// Função para medir o tempo e exibir uso de memória
void display_gpu_memory_info() {
    size_t free_mem, total_mem;
    cudaMemGetInfo(&free_mem, &total_mem);
    std::cout << "Memória total da GPU: " << total_mem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre na GPU: " << free_mem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória usada pela GPU: " << (total_mem - free_mem) / (1024 * 1024) << " MB" << std::endl;
}

int main() {
    const int N = 1 << 20;  // Tamanho do vetor (1 milhão de elementos)

    // Gerar dados aleatórios
    thrust::device_vector<float> d_vec(N);
    fill_vector(d_vec);  // Preencher o vetor com valores

    // Exibir informações iniciais da GPU
    display_gpu_memory_info();

    // Cálculo da média
    auto start_mean = std::chrono::high_resolution_clock::now();
    float mean = calculate_mean(d_vec);
    auto end_mean = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> mean_duration = end_mean - start_mean;

    std::cout << "Média calculada: " << mean << std::endl;
    std::cout << "Tempo de cálculo da média: " << mean_duration.count() << " segundos" << std::endl;

    // Variância em etapas separadas
    auto start_separate = std::chrono::high_resolution_clock::now();
    float variance_separate = calculate_variance_separate(d_vec, mean);
    auto end_separate = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> separate_duration = end_separate - start_separate;

    std::cout << "Variância (etapas separadas): " << variance_separate << std::endl;
    std::cout << "Tempo de cálculo da variância (etapas separadas): " << separate_duration.count() << " segundos" << std::endl;

    // Variância usando Fusion Kernel
    auto start_fusion = std::chrono::high_resolution_clock::now();
    float variance_fusion = calculate_variance_fusion(d_vec, mean);
    auto end_fusion = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> fusion_duration = end_fusion - start_fusion;

    std::cout << "Variância (Fusion Kernel): " << variance_fusion << std::endl;
    std::cout << "Tempo de cálculo da variância (Fusion Kernel): " << fusion_duration.count() << " segundos" << std::endl;

    // Exibir informações finais da GPU
    display_gpu_memory_info();

    return 0;
}


Overwriting variance_fusion.cu


In [32]:
!nvcc -arch=sm_75 variance_fusion.cu -o variance_fusion

In [33]:
!./variance_fusion

Memória total da GPU: 15102 MB
Memória livre na GPU: 14993 MB
Memória usada pela GPU: 109 MB
Média calculada: 524288
Tempo de cálculo da média: 0.000615758 segundos
Variância (etapas separadas): 9.1626e+10
Tempo de cálculo da variância (etapas separadas): 0.00065318 segundos
Variância (Fusion Kernel): 9.1626e+10
Tempo de cálculo da variância (Fusion Kernel): 0.00030065 segundos
Memória total da GPU: 15102 MB
Memória livre na GPU: 14993 MB
Memória usada pela GPU: 109 MB


# Interpretação dos Resultados da Atividade 12

## Exercício 1 (SAXPY)
A operação de SAXPY, que combina vetores utilizando uma constante multiplicadora, mostrou-se altamente eficiente ao ser executada na GPU. O tempo de execução foi extremamente curto (aproximadamente 3.57e-05 segundos), evidenciando a capacidade da GPU de lidar com operações paralelas em vetores. A memória usada pela GPU permaneceu constante durante a execução, indicando que o processamento não exigiu grande alocação adicional de memória.

## Exercício 2 (Cálculo da Magnitude)
Ao comparar a execução do cálculo da magnitude entre a GPU e a CPU, observou-se que o tempo de execução na GPU foi significativamente menor (0.000445143 segundos na GPU contra 0.0109731 segundos na CPU). Isso reflete o ganho de desempenho substancial ao utilizar o processamento paralelo na GPU para operações matemáticas intensivas. Além disso, o uso de memória da GPU foi consistente, sem grandes variações, o que demonstra eficiência no gerenciamento de recursos pela GPU.

## Exercício 3 (Cálculo da Variância com Fusion Kernel)
O uso do Fusion Kernel para calcular a variância permitiu combinar várias operações em um único kernel, resultando em um tempo de execução mais rápido (0.00030065 segundos) em comparação com a abordagem tradicional, que executa as operações em etapas separadas (0.00065318 segundos). Embora a diferença de tempo pareça pequena, em grandes volumes de dados e operações mais complexas, essa otimização pode gerar ganhos substanciais. O uso da memória da GPU manteve-se eficiente durante todo o processo.

## Conclusão
O uso de operações paralelas na GPU proporciona uma vantagem significativa em termos de tempo de execução, especialmente em tarefas matemáticas e de processamento de grandes vetores. A técnica de Fusion Kernel, em particular, mostrou-se eficaz na otimização de operações, reduzindo o tempo de execução ao combinar cálculos que seriam realizados separadamente. Além disso, o uso eficiente da memória da GPU durante todos os exercícios confirma a adequação dessas técnicas para o processamento em larga escala.
